In [1]:
from astropy.coordinates import SkyCoord
from astropy.coordinates.name_resolve import NameResolveError
from astropy.table import Table, Row, vstack, unique, hstack
from astropy.time import Time
from astropy import units as u

from astroquery.vizier import Vizier

import numpy

from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm

from pathlib import Path

import itertools
import pickle

from targets import ESCAPED_TARGET_NAMES
import matching

In [2]:
DATA_PATH = Path("//stem-linux-homes/OSL-Telescope/data/users/Pipeline/")
NUM_CALIBRATION_ROWS = 10
CALIBRATION_CATALOGUES = {
    'I/284/out': {
        'B': 'B1mag',
        'R': 'R1mag',
        'I': 'Imag',
    },
    'I/297/out': {
        'B': 'Bmag',
        'V': 'Vmag',
        'R': 'Rmag',
    },
    'VI/135/table15': {
        'B': 'BTmag',
        'V': 'VTmag',
    },
    'II/339/uvotssc1': {
        'U': 'Umag',
        'B': 'Bmag',
        'V': 'Vmag',
    }
}
REPROCESS = True # Set to True to reprocess previous data rather than finding new data

In [3]:
try: 
    with open('data/processed_dates.pickle', 'rb') as processed_dates_file:
        processed_dates = pickle.load(processed_dates_file)
except FileNotFoundError:
    processed_dates = {}

In [4]:
if REPROCESS:
    processed_dates = [ d for d, has_data in processed_dates.items() if not has_data ]

In [5]:
new_dates = [p for p in DATA_PATH.glob('*/202?_??_??') if p not in processed_dates]
total_dates = len(new_dates)
obs_tables = {}

SPLIT_SIZE = 10

for date in tqdm(new_dates, desc='New dates'):
    date_has_data = False
    for catalogue in tqdm(list(date.glob('Catalogues/*_anm83_*.cat')), desc=str(date)):
        date_has_data = True
        name = None
        for escaped_target_name, target_name in ESCAPED_TARGET_NAMES.items():
            if escaped_target_name in catalogue.stem:
                name = target_name
                obs_meta = list(itertools.chain.from_iterable(l.split('_') for l in catalogue.stem.split(f'_{escaped_target_name}_')))
                break
        if name is None:
            continue
        obs_meta = {
            'telescope': obs_meta[0],
            'main target': name,
            'band': obs_meta[6][0],
            'exposure': float(obs_meta[6][1:]),
            'timestamp': Time(
                dict(zip(
                    ['year', 'month', 'day', 'hour', 'minute', 'second'],
                    map(int, obs_meta[8:14])
                )),
                format='ymdhms',
            ).jd,
        }

        
        table = Table.read(catalogue, format='ascii.sextractor')
        table.rename_column('ALPHA_J2000', 'RA')
        table.rename_column('DELTA_J2000', 'Dec')

        matching.match_targets(table)
        table.rename_column('RA', '_RAJ2000')
        table.rename_column('Dec', '_DEJ2000')
        matched_targets = table[table['matched target'].mask == False]
        
        for target_row in matched_targets:
            out_table = Table(target_row)

            calibration_succeeded = False

            Path(f"data/calibration_tables/{target_row['matched target']}").mkdir(parents=True, exist_ok=True)
            for calibration_catalogue, calibration_catalogue_fields in CALIBRATION_CATALOGUES.items():
                if obs_meta['band'] in calibration_catalogue_fields:
                    calibration_catalogue_path = f"data/calibration_tables/{target_row['matched target']}/{calibration_catalogue.replace('/', '_')}_{catalogue.stem}.ecsv"
                    try:
                        calibration_table = Table.read(calibration_catalogue_path)
                    except FileNotFoundError:
                        table = table[table['FLAGS'] == 0]
                        table['flux_diff'] = numpy.abs(table['FLUX_AUTO'] - target_row['FLUX_AUTO'])
                        table.sort('flux_diff')
                        table = table[:50]
                        table['separation'] = table['coords'].separation(target_row['coords'])
                        table.sort('separation')

                        calibration_rows = []

                        total_iterations = int(len(table) / SPLIT_SIZE) + 1
                        for i in range(total_iterations):
                            sources = table[i * SPLIT_SIZE : (i+1) * SPLIT_SIZE]
                            if len(sources) == 0:
                                continue
                            sources = vstack([ row for row in sources if row['NUMBER'] != target_row['NUMBER'] ])
                            catalogue_results = Vizier.query_region(sources, radius=1e-4*u.deg, catalog=calibration_catalogue)
                            if len(catalogue_results) == 0:
                                continue

                            # Reject any sources with multiple matches
                            catalogue_matches = unique(catalogue_results[0], '_q', keep='none')

                            # To do: Reject any known variables

                            for catalogue_row in catalogue_matches[~numpy.isnan(catalogue_matches[calibration_catalogue_fields[obs_meta['band']]])]:
                                calibration_rows.append(hstack([
                                    sources[int(catalogue_row['_q']) - 1],
                                    catalogue_row[[calibration_catalogue_fields[obs_meta['band']]]],
                                ]))

                            if len(calibration_rows) >= NUM_CALIBRATION_ROWS:
                                break
                        if len(calibration_rows) < NUM_CALIBRATION_ROWS:
                            continue
                        calibration_table = vstack(calibration_rows[:NUM_CALIBRATION_ROWS], metadata_conflicts='silent')
                        calibration_table.write(
                            calibration_catalogue_path,
                            overwrite=True,
                        )

                    calibrated_mags = (
                        calibration_table[calibration_catalogue_fields[obs_meta['band']]] 
                        - 2.5 * numpy.log10(target_row['FLUX_AUTO'] / calibration_table['FLUX_AUTO'])
                    )

                    # To do: Maybe this should be a weighted average using the flux error?
                    out_table.add_columns(
                        [
                            numpy.mean(calibrated_mags),
                            # Standard error as per https://www.statology.org/standard-error-of-mean-python/
                            numpy.std(calibrated_mags, ddof=1) / numpy.sqrt(numpy.size(calibrated_mags)),
                            calibration_catalogue,
                            obs_meta['main target'],
                        ],
                        names=[
                            'calibrated_mag',
                            'calibrated_mag_err',
                            'calibration_catalogue',
                            'main_target',
                        ],
                    )
                    calibration_succeeded = True
                    break

            if not calibration_succeeded:
                out_table.add_columns(
                    [
                        numpy.nan,
                        numpy.nan,
                        '',
                        obs_meta['main target'],
                    ],
                    names=[
                        'calibrated_mag',
                        'calibrated_mag_err',
                        'calibration_catalogue',
                        'main_target',
                    ],
                )

            for key, val in obs_meta.items():
                out_table[key] = val

            if target_row['matched target'] not in obs_tables:
                try:
                    if REPROCESS:
                        obs_tables[target_row['matched target']] = out_table
                        continue
                    else:
                        obs_tables[target_row['matched target']] = Table.read(f"data/{target_row['matched target']}.ecsv")
                except FileNotFoundError:
                    obs_tables[target_row['matched target']] = out_table
                    continue
            obs_tables[target_row['matched target']] = vstack([obs_tables[target_row['matched target']], out_table])
        
    if not REPROCESS:
        processed_dates[date] = date_has_data

New dates:   0%|          | 0/57 [00:00<?, ?it/s]

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_09_18:   0%|          | 0/27 [00:00<?, ?it/s]

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_09_21:   0%|          | 0/27 [00:00<?, ?it/s]

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_10_28:   0%|          | 0/8 [00:00<?, ?it/s]

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_10_29:   0%|          | 0/8 [00:00<?, ?it/s]

...

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_10_31:   0%|          | 0/29 [00:00<?, ?it/s]

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_11_02:   0%|          | 0/6 [00:00<?, ?it/s]

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_11_03:   0%|          | 0/19 [00:00<?, ?it/s]

\\stem-linux-homes\OSL-Telescope\data\users\Pipeline\COAST\2021_11_05:   0%|          | 0/14 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
for name, table in obs_tables.items():
    table.write(f"data/{name}.ecsv", overwrite=True)

In [ ]:
if not REPROCESS:
    with open('data/processed_dates.pickle', 'wb') as processed_dates_file:
        pickle.dump(processed_dates, processed_dates_file)